In [1]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import pandas as pd

In [2]:
df=pd.read_csv("adult.csv")

In [3]:
df

,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32555,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32556,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32557,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32558,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [4]:
df.shape

(32560, 15)

In [5]:
le = LabelEncoder()

In [6]:
ds = df.apply(le.fit_transform)

In [7]:
ds

,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
0,33,6,2925,9,12,2,4,0,4,1,0,0,12,39,0
1,21,4,14085,11,8,0,6,1,4,1,0,0,39,39,0
2,36,4,15335,1,6,2,6,0,2,1,0,0,39,39,0
3,11,4,19354,9,12,2,10,5,2,0,0,0,39,5,0
4,20,4,17699,12,13,2,4,5,4,0,0,0,39,39,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32555,10,4,16527,7,11,2,13,5,4,0,0,0,37,39,0
32556,23,4,8079,11,8,2,7,0,4,1,0,0,39,39,1
32557,41,4,7882,11,8,6,1,4,4,0,0,0,39,39,0
32558,5,4,12880,11,8,4,1,3,4,1,0,0,19,39,0


In [8]:
data = ds.values
print(data.shape)
print(type(data))

(32560, 15)
<class 'numpy.ndarray'>


In [9]:
data[:5,:]

array([[   33,     6,  2925,     9,    12,     2,     4,     0,     4,
            1,     0,     0,    12,    39,     0],
       [   21,     4, 14085,    11,     8,     0,     6,     1,     4,
            1,     0,     0,    39,    39,     0],
       [   36,     4, 15335,     1,     6,     2,     6,     0,     2,
            1,     0,     0,    39,    39,     0],
       [   11,     4, 19354,     9,    12,     2,    10,     5,     2,
            0,     0,     0,    39,     5,     0],
       [   20,     4, 17699,    12,    13,     2,     4,     5,     4,
            0,     0,     0,    39,    39,     0]])

In [10]:
data_X = data[:, :14]
data_Y = data[:, -1] #check

In [20]:
X_train, X_test, Y_train, Y_test = train_test_split(data_X, data_Y, test_size = 0.2,random_state= 38)

In [21]:
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((26048, 14), (6512, 14), (26048,), (6512,))

In [22]:
np.unique(Y_train)

array([0, 1])

In [23]:
def prior_prob(y_train, label):
    return np.sum(y_train == label)/float(y_train.shape[0])

In [24]:
def cond_prob(X_train, Y_train, feature_col, feature_val, label):

    X_train = X_train[Y_train == label]
    numerator_mushrooms = np.sum(X_train[:, feature_col] == feature_val)
    denominator_mushrooms = np.sum(Y_train == label)

    return numerator_mushrooms/float(denominator_mushrooms)

In [25]:
def predict(X_train, Y_train, X_test):
    classes = np.unique(Y_train)

    n_feat = X_train.shape[1]
    post_prob = []

    for label in classes:
        likelihood = 1.0

        for f in range(n_feat):
            conditional_prob = cond_prob(X_train, Y_train, f, X_test[f], label)
            likelihood *= conditional_prob

        prior = prior_prob(Y_train, label)

        posterior = likelihood*prior
        post_prob.append(posterior)

    pred = np.argmax(post_prob)

    return pred

In [26]:
output = predict(X_train, Y_train, X_test[1])
print(output, Y_test[1])  

0 0


In [27]:
def score(X_train, X_test, Y_train, Y_test):
    pred = []

    for i in range(X_test.shape[0]):
        predict_label = predict(X_train, Y_train, X_test[i])

        pred.append(predict_label)

    pred = np.array(pred)

    accuracy = np.sum(pred == Y_test)/Y_test.shape[0]

    return accuracy

In [28]:
print(score(X_train, X_test, Y_train, Y_test))

0.7820945945945946
